# Podcast Episode Erstellen

### imports

In [ ]:
import sys
sys.path.append(".")
from audio_transcription.faster_whisper_word_level import transcribe
from Episodes_Downloader.episodes_downloader import get_names_and_urls_all_episodes, download_and_save_mp3_in_dir
import os
import pandas as pd
from pydub import AudioSegment
from pydub.playback import play
from playsound import playsound
import IPython 
import io
from db_connect import db_get_df,db_save_df
from segment_ranking.rank_segments import get_most_similar_documents_Llama2
from Audio_segmentation.split_audio import produce_snippets
from Audio_segmentation.concat_audio import produce_audio
from tqdm import tqdm

### Episoden URLs laden

In [ ]:
titles, audio_urls = get_names_and_urls_all_episodes()
filenames = [url.split("/")[-1] for url in audio_urls]
print(titles)

In [ ]:
print(filenames)

In [ ]:
df = db_get_df("transcripts")
df.dtypes

In [ ]:
df = pd.DataFrame({"title": titles, "download_url": audio_urls, "transcript": None, "filename":filenames})
db_save_df(df, "transcripts_all")

### Alle mp3s herunterladen

In [ ]:
df = db_get_df("transcripts_all")
path = "/nfs/scratch/students/neumannvi84434/Podcast_Episoden/"
for i, row in tqdm(df.iterrows()):
    title = row["title"].replace("/","")
    if title in os.listdir(path):
        continue
    download_and_save_mp3_in_dir(row["download_url"], path, title)


In [ ]:
directory_path = "/nfs/scratch/students/neumannvi84434/Podcast_Episoden/"

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    old_filename = row["title"]
    new_filename = row["filename"]
    
    # Construct the full paths for the old and new filenames
    old_file_path = os.path.join(directory_path, old_filename)
    new_file_path = os.path.join(directory_path, new_filename)
    
    try:
        # Rename the file using os.rename
        os.rename(old_file_path, new_file_path)
        print(f"Renamed: {old_filename} to {new_filename}")
    except FileNotFoundError:
        print(f"File not found: {old_filename}")
    except FileExistsError:
        print(f"File already exists: {new_filename}")
    except Exception as e:
        print(f"An error occurred while renaming {old_filename} to {new_filename}: {e}")


In [ ]:
df = db_get_df("transcripts_all")
path = "/nfs/scratch/students/neumannvi84434/Podcast_Episoden/"
print(len(df))
print(len(os.listdir(path)))
# welche nicht als file abgespiechert wurden
print(len(df[~df['title'].isin(os.listdir(path))]))
print(len(df[df.duplicated(subset='title', keep=False)]))
print(df[df.duplicated(subset='title', keep=False)].sort_values(by="title").to_markdown())


In [ ]:
df.loc[179, "download_url"]

### Transkription auf Word level Ebene

In [ ]:
df = db_get_df("transcripts_all")
df_words = pd.DataFrame({"filename": [], 
                          "index": [],
                          "word": [],
                          "start": [],
                          "end": []})

path = "/nfs/scratch/students/neumannvi84434/Podcast_Episoden/"

for filename in os.listdir(path):
    transcript_df = transcribe(os.path.join(path,filename))
    df_words = pd.concat([df_words, transcript_df])

In [ ]:
df = transcribe("/Users/br/Projects/Bachelorarbeit/data/Episode_audio_files/jonathan swift gullivers reisen 2.mp3")
# db_save_df(df,"transcript_word_level")

In [ ]:
db_save_df(df, "transcript_gulliver_word_level")

### debug db

In [ ]:
import sqlite3
import pandas as pd

con = sqlite3.connect("/nfs/scratch/students/neumannvi84434/backup/transcripts.sqlite")
df = pd.read_sql_query(f"SELECT * FROM transcript_word_level_2237", con)
con.close()
df.dtypes

In [ ]:
len(df)

### Satzbildung mit spacy

In [ ]:
import spacy

In [ ]:
df = db_get_df("transcript_gulliver_word_level")
df.dtypes

Zunächst erstmal mit Whispers Punkten

In [ ]:
print(f'|{df["word"][2]}|')

In [ ]:
sentences = []
one_sentence = []
for word in df["word"]:
    one_sentence.append(word.strip())
    if "." in word:
        sentences.append(" ".join(one_sentence))
        one_sentence = []
print(sentences)

### Demonstration

In [ ]:
df = db_get_df(table="transcripts")

In [ ]:
userInputText = "Auf dem l Feiern und trinken"
userInputTime = 7
best_fitting = get_most_similar_documents_Llama2(userInputText, userInputTime)
print(best_fitting["segment_text"].to_markdown())

produce_snippets()
produce_audio()


In [ ]:
IPython.display.Audio("concatenated_audio.mp3")